# Exploring GPT: Section detailing how to connect

## Using the API

First, you'll need install the OpenAPI via pip.  You can use Unix commands in a colab notebook by prefixing them with an exclamation point. The example below uses a `%` prefix instead—this guarantees that the installation is done using the same Python version that your notebook is using. Change line 2 from `%pip...` to `!pip...` if you're running into trouble.

In [1]:
%%capture
%pip install openai

Next, you will enter your secret key for the OpenAI API. You can generate your OpenAI API key [here](https://platform.openai.com/api-keys). Also, start the service through the client variable.

In [2]:
from getpass import getpass
import openai
import os
import time


print('Enter OpenAI API key:')
openai.api_key = getpass()

os.environ['OPENAI_API_KEY']=openai.api_key

client = openai.OpenAI()

Enter OpenAI API key:


# Experimentation, defining the API function calls for...

### 1. City activity finder
### 2. Meal Prep Generator





In [38]:
def city_activity(city):
  zero_shot_prompt = """Output a list of four reccomendations, line by line.
                      """

  # All GPT completions are generated by the chat model now. The chat model works best
  # when it's given a baseline instruction about how GPT should behave. We'll use this one.
  system_message = {"role" : "system", "content" : "Give a list of four activites to do in the given city, line by line."}

  client = openai.OpenAI()
  response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages= [
          system_message,
          {"role" : "user", "content" : zero_shot_prompt + city + " - "} # simulate a user prompt
      ],
      temperature=0.7,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      stop=["---"]
  )
  
  time.sleep(1)

  # the response from OpenAI's API is a JSON object that contains the completion to your prompt plus some other information.  
  # Here's how to access just the text of the completion.
  list = response.choices[0].message.content.strip()
  return list

#######################
#######################
#######################
#######################

def meal_plan(allergies, fitness_goal):
  # TODO - write this function
  zero_shot_prompt = """Given the dietary restrictions and fitness goals, output a list of what to eat during the week.
                      """

  # All GPT completions are generated by the chat model now. The chat model works best
  # when it's given a baseline instruction about how GPT should behave. We'll use this one.
  system_message = {"role" : "system", "content" : "Output a list of what to eat in the week based on the dietary restrictions and fitness goals."}

  client = openai.OpenAI()
  response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages= [
          system_message,
          {"role" : "user", "content" : zero_shot_prompt + allergies + fitness_goal + " - "} # simulate a user prompt
      ],
      temperature=0.7,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      stop=["---"]
  )
  
  time.sleep(1)

  # the response from OpenAI's API is a JSON object that contains
  # the completion to your prompt plus some other information.  Here's how to access
  # just the text of the completion.
  summary = response.choices[0].message.content.strip()
  return summary



# Running the functions

In [40]:
city = """
Denver
"""

answer = city_activity(city)
print(answer)

1. Visit the Red Rocks Amphitheatre for a concert or hike among the stunning rock formations.
2. Explore the Denver Art Museum to see a diverse collection of art from around the world.
3. Take a stroll through the historic Larimer Square for shopping, dining, and vibrant nightlife.
4. Head to Denver Botanic Gardens to admire beautiful plant displays and peaceful gardens.


In [39]:
allergies = """
Peanuts, strawberries, gluten.
"""
fitness_goals = """
I would like to gain 20 pounds of muscle in the next year. 
"""


answer = meal_plan(allergies, fitness_goals)
print(answer)
  


Based on your dietary restrictions of avoiding peanuts, strawberries, and gluten, here is a sample meal plan for the week to help you gain muscle:

Monday:
- Breakfast: Scrambled eggs with spinach and avocado
- Snack: Greek yogurt with honey and almonds
- Lunch: Grilled chicken with quinoa and roasted vegetables
- Snack: Carrot sticks with hummus
- Dinner: Baked salmon with sweet potato and asparagus

Tuesday:
- Breakfast: Oatmeal with almond butter and banana
- Snack: Apple slices with almond butter
- Lunch: Turkey and avocado wrap with lettuce
- Snack: Rice cakes with tuna salad
- Dinner: Beef stir-fry with broccoli and brown rice

Wednesday:
- Breakfast: Smoothie with spinach, banana, protein powder, and almond milk
- Snack: Cottage cheese with pineapple
- Lunch: Lentil soup with a side salad
- Snack: Celery sticks with peanut butter (if tolerated)
- Dinner: Grilled shrimp with quinoa and steamed green beans

Thursday:
- Breakfast: Greek yogurt with granola and mixed berries (exclud

# Final Thoughts


That's it!  That's an example of how to write a function call to the OpenAI API in order for it to output a subject for a topic.

Here is some information about the different arguments that we to the `openai.Completion.create` call:
 * `model` – OpenAI offers many different versions of their chat model. We'll use GPT3.5-turbo, which is functional without being [the most expensive to run](https://openai.com/api/pricing/).
 * `messages` - this is the set of messages that will instruct how the model should respond. The first message is a system prompt—an overall setting that specifies how the model should behave. In this case, the second message is the "training set" of examples followed by the last topic that we want to generate a subject for.
 * `temperature` - controls how much of the probability distribution the model will use when it is generating each token. 1.0 means that it samples from the complete probability distrubiton, 0.7 means that it drops the bottom 30% of the least likely tokens when it is sampling. 0.0 means that it will perform deterministically and always output the single most probable token for each context.
 * `top_p` - is an alternative way of controling the sampling.
 * `frequency_penalty` and `presence_penalty` are two ways of reduing the model from repeating the same words in one output.  You can set these to be >0 if you're seeing a lot of repetition in your output.
 * `max_tokens` is the maximum length in tokens that will be output by calling the function.  A token is a subword unit.  There are roughly 2 or 3 tokens per word on average.
 * `stop` is a list of stop sequences.  The model will stop generating output once it generates one of these strings, even if it hasn't reached the max token length. By default this is set to a special token `<|endoftext|>`.

You can read more about [the Chat Completion API call in the documentation](https://cookbook.openai.com/examples/how_to_format_inputs_to_chatgpt_models).